## 3.1 Churn prediction Project

3.2 Data Preparation

In [1]:
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
data='https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [ ]:
!wget $data -O data-week-3.csv 

In [ ]:
df = pd.read_csv('data-week-3.csv')
df.head()

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [ ]:
df.head().T

In [ ]:
df.dtypes

In [ ]:
tc = pd.to_numeric(df.totalcharges, errors='coerce')

In [ ]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')

In [ ]:
df.totalcharges = df.totalcharges.fillna(0)

In [ ]:
df.churn.head()

In [ ]:
df.churn = (df.churn == 'yes').astype(int)

## 3.3 Setting up the validation framework 

Perform the train/validation/test split with Scikit-Learn


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [ ]:
len(df_train), len(df_val), len(df_test)

In [ ]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

del df_train['churn']
del df_val['churn']
del df_test['churn']

## 3.4 EDA
    Check missing values
    Look at the target variable (churn)
    Look at numerical and categorical variables

In [ ]:
df_full_train = df_full_train.reset_index(drop=True)

In [ ]:
df_full_train.churn.value_counts(normalize=True)

In [ ]:
global_churn_rate = df_full_train.churn.mean()
round(global_churn_rate, 2)

In [ ]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [ ]:
df_full_train.columns

In [ ]:
categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [ ]:
df_full_train[categorical].nunique()

## 3.5 Feature importance: Churn rate and risk ratio

In [ ]:
df_full_train.head()

Churn rate

In [ ]:
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

In [ ]:
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

In [ ]:
global_churn = df_full_train.churn.mean()
global_churn

In [ ]:
df_full_train.partner.value_counts()

In [ ]:
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_partner 

In [ ]:
global_churn - churn_partner

In [ ]:
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean()
churn_no_partner

In [ ]:
# 

In [ ]:
global_churn - churn_no_partner

Risk Ratio

In [ ]:
churn_no_partner / global_churn

In [ ]:
churn_partner / global_churn

In [ ]:
df_full_train.groupby('gender').churn.mean()

In [ ]:
from IPython.display import display

In [ ]:
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    df_group
    display(df_group)
    print()
    print()

## 3.6 Feature importance: Mutual information

Mutual information - concept from information theory, it tells us how much we can learn about one variable if we know the value of another

In [ ]:
from sklearn.metrics import mutual_info_score

In [ ]:
mutual_info_score(df_full_train.churn, df_full_train.contract)

mutual_info_score(df_full_train.churn, df_full_train.contract)

In [ ]:
mutual_info_score(df_full_train.churn, df_full_train.gender)

In [ ]:
mutual_info_score(df_full_train.churn, df_full_train.partner)

In [ ]:
mutual_info_score(df_full_train.churn, df_full_train.contract)

In [ ]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [ ]:
ml = df_full_train[categorical].apply(mutual_info_churn_score)
ml.sort_values(ascending=False)

## 3.7 Feature importance: Correlation

In [ ]:
df_full_train.tenure.max()

In [ ]:
df_full_train[numerical].corrwith(df_full_train.churn).abs()

In [ ]:
df_full_train[df_full_train.tenure <= 2].churn.mean()

In [ ]:
df_full_train[(df_full_train.tenure > 2) & (df_full_train.tenure <= 12)].churn.mean()

In [ ]:
df_full_train[df_full_train.tenure > 12].churn.mean()

In [ ]:
df_full_train[df_full_train.monthlycharges <= 20].churn.mean()

In [ ]:
df_full_train[(df_full_train.monthlycharges > 20) & (df_full_train.monthlycharges <= 50)].churn.mean()

In [ ]:
df_full_train[df_full_train.monthlycharges > 50].churn.mean()

## 3.8 One-hot encoding

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

## 3.9 Logistic regression

    Binary classification
    Linear vs logistic regression



In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
z = np.linspace(-5, 5, 51)

In [ ]:
sigmoid(z)

In [ ]:
 plt.plot(z, sigmoid(z))

In [ ]:
def linear_regression(xi):
    result = w0

    for j in range(len(w)):
        score = score + xi[j] * w[j]

    return result

In [ ]:
def logistic_regression(xi):
    result = w0

    for j in range(len(w)):
        score = score + xi[j] * w[j]

    result = sigmoid(score)
    return result

## 3.10 Training logistic regression with Scikit-Learn

    Train a model with Scikit-Learn
    Apply it to the validation dataset
    Calculate the accuracy



In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(max_iter=1000, solver='liblinear')
model.fit(X_train, y_train)

In [ ]:
model.coef_[0].round(3)

In [ ]:
y_pred = model.predict_proba(X_val)[:, 1]

In [ ]:
churn_decision = (y_pred >= 0.5)

(y_val == churn_decision).mean() 

In [ ]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = churn_decision.astype(int)
df_pred['actual'] = y_val

In [ ]:
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [ ]:
df_pred.correct.mean()

In [ ]:
churn_decision.astype(int)

## 3.11 Model interpretation

    Look at the coefficients
    Train a smaller model with fewer features



In [ ]:
a = [1, 2, 3, 4]
b = 'abcd'

In [ ]:
dict(zip(a, b))

In [ ]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

In [ ]:
small = ['contract', 'tenure', 'monthlycharges']

In [ ]:
df_train[small].iloc[:10].to_dict(orient='records')

In [ ]:
dicts_train_small = df_train[small].to_dict(orient="records")
dicts_val_small = df_val[small].to_dict(orient="records")

In [ ]:
dv_small = DictVectorizer(sparse=False)
dv_small.fit(dicts_train_small)

In [ ]:
dv_small.get_feature_names_out()

In [ ]:
X_train_small = dv_small.transform(dicts_train_small)

In [ ]:
model_small = LogisticRegression()
model_small.fit(X_train_small, y_train)

In [ ]:
w0 = model_small.intercept_[0]
w0

In [ ]:
w = model_small.coef_[0]
w.round(3)

In [ ]:
dict(zip(dv_small.get_feature_names_out(), model.coef_[0].round(3)))

In [ ]:
-2.47 + 0.97 + 50 * 0.028 + 5 * (0.036)

In [ ]:
sigmoid(_)

## Using the model

In [ ]:
dicts_full_train = df_full_train[categorical + numerical].to_dict(orient='records')

In [ ]:
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)

In [ ]:
y_full_train = df_full_train.churn.values

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_full_train, y_full_train)

In [ ]:
dicts_test = df_test[categorical +  numerical].to_dict(orient='records')

In [ ]:
X_test = dv.transform(dicts_test)

In [ ]:
y_pred = model.predict_proba(X_test)[:, 1]

In [ ]:
churn_decision = (y_pred >= 0.5) 

In [ ]:
(churn_decision == y_test).mean()

In [ ]:
customer = dicts_test[520]
customer

In [ ]:
X_small = dv.transform([customer])

In [ ]:
model.predict_proba(X_small)[0, 1]

In [ ]:
y_test[10]